# Import `Recomender` class

In [1]:
from tfrec import Recommender

ModuleNotFoundError: No module named 'tensorflow'

# A Small Dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
M = np.array([[4, 3, 5, 2, 1],
              [3, 5, 5, 1, 2],
              [4, 4, 4, 4, 4],
              [5, 5, 5, 5, 5],
              [0, 2, 0, 4, 4],
              [1, 0, 0, 5, 5],
              [0, 1, 1, 4, 3]])
data_matrix = pd.DataFrame(M, index=['ryan', 'scott g', 'annie', 'scott s', 'nicole', 'carly', 'bill'], columns=['Matrix', 'Alien', 'StarWars', 'Casablanca', 'Titanic'])
data_matrix

,Matrix,Alien,StarWars,Casablanca,Titanic
ryan,4,3,5,2,1
scott g,3,5,5,1,2
annie,4,4,4,4,4
scott s,5,5,5,5,5
nicole,0,2,0,4,4
carly,1,0,0,5,5
bill,0,1,1,4,3


**Note:** Zeros above denote missing ratings.

# Transform the Dataset

In [4]:
users = []
movies = []
ratings = []

for name, row in data_matrix.iterrows():
    for movie, rating in row.iteritems():
        if rating > 0:
            users.append(name)
            movies.append(movie)
            ratings.append(rating)
            
data_ratings = pd.DataFrame({'user': users, 'movie': movies, 'rating': ratings}, columns=['user', 'movie', 'rating'])
data_ratings

,user,movie,rating
0,ryan,Matrix,4
1,ryan,Alien,3
2,ryan,StarWars,5
3,ryan,Casablanca,2
4,ryan,Titanic,1
5,scott g,Matrix,3
6,scott g,Alien,5
7,scott g,StarWars,5
8,scott g,Casablanca,1
9,scott g,Titanic,2


# Train The Model!

**Note:** Since this is a small, silly example WE'RE NOT DOING CROSS VALIDATION! This is just an example of how to use the model.

In [5]:
model = Recommender(k=8,
                    dtype='float32',
                    lambda_factors=0.0001,
                    lambda_biases=0.0001,
                    init_factor_mean=0.0,
                    init_factor_stddev=0.01,
                    n_iter=5000,
                    learning_rate=1e-03,
                    batch_size=-1)

X = data_ratings[['user', 'movie']].values
y = data_ratings['rating'].values

model.fit(X, y, verbose=True, verbose_period=1000)

2017-05-15 00:19:40,388: tfrec.recommender : INFO      : will `fit()` fresh
2017-05-15 00:19:40,392: tfrec.recommender : INFO      : new_num_users: 9, new_num_items: 7
2017-05-15 00:19:40,393: tfrec.recommender : INFO      : num_users: 9, num_items: 7
2017-05-15 00:19:41,476: tfrec.recommender : INFO      : instantiated a new TensorFlow session
2017-05-15 00:19:41,779: tfrec.recommender : INFO      : Starting Gradient Descent for 5000 iterations
2017-05-15 00:19:41,789: tfrec.recommender : INFO      : training set RMSE = 1.4545047283172607
2017-05-15 00:19:41,840: tfrec.recommender : INFO      : Finished iteration #1
2017-05-15 00:19:41,849: tfrec.recommender : INFO      : training set RMSE = 1.449751853942871
2017-05-15 00:19:44,315: tfrec.recommender : INFO      : Finished iteration #1001
2017-05-15 00:19:44,317: tfrec.recommender : INFO      : training set RMSE = 0.2630063593387604
2017-05-15 00:19:46,736: tfrec.recommender : INFO      : Finished iteration #2001
2017-05-15 00:19:46,

Recommender(batch_size=-1, dtype='float32', init_factor_mean=0.0,
      init_factor_stddev=0.01, k=8, lambda_biases=0.0001,
      lambda_factors=0.0001, learning_rate=0.001, n_iter=5000)

In [6]:
new_user = [('Matrix', 5), ('StarWars', 3), ('Casablanca', 1)]
new_user_predictions = model.predict_new_user(new_user, n_iter=1000)
new_user_predictions

array([ 2.63598251,  2.63598251,  4.00939083,  2.8446486 ,  3.71903157,
        1.45365047,  1.42439508], dtype=float32)

In [7]:
model.index_to_item_map_

{0: '__unknown__',
 1: '__new_entry__',
 2: 'Matrix',
 3: 'Alien',
 4: 'StarWars',
 5: 'Casablanca',
 6: 'Titanic'}

In [8]:
ordered_movies = sorted([(val, model.index_to_item_map_[i]) for i, val in enumerate(new_user_predictions)], reverse=True)
ordered_movies

[(4.0093908, 'Matrix'),
 (3.7190316, 'StarWars'),
 (2.8446486, 'Alien'),
 (2.6359825, '__unknown__'),
 (2.6359825, '__new_entry__'),
 (1.4536505, 'Casablanca'),
 (1.4243951, 'Titanic')]

# Visualize with TensorBoard

In [9]:
import tensorflow as tf
writer = tf.summary.FileWriter('tfout')
writer.add_graph(model.sess.graph)

Now, run the command:

```bash
> tensorboard --logdir tfout/
```